# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
C:\Users\IBM\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [3]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [5]:
df = raw_df.copy()
df['combined_text'] = df.headline + ' ' +  df.short_description
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [6]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [7]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [8]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55564026, 67336880)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [9]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [10]:
wv.most_similar('Texas')

C:\Users\IBM\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Pennsylvania', 0.8130167126655579),
 ('Ohio', 0.7915807366371155),
 ('Louisiana', 0.7823953628540039),
 ('Oklahoma', 0.7823035717010498),
 ('Georgia', 0.7794561386108398),
 ('Illinois', 0.777507483959198),
 ('Florida', 0.7712527513504028),
 ('Maryland', 0.7682380080223083),
 ('Oregon', 0.7671371698379517),
 ('Connecticut', 0.7655514478683472)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [11]:
wv.most_similar(negative='Texas')

[('once-reliable', 0.41439443826675415),
 ('maitre', 0.41407978534698486),
 ('End-of-School', 0.38328906893730164),
 ('Parent/Grandparent', 0.38172340393066406),
 ('Likened', 0.3803216218948364),
 ('wall-to-wall', 0.37504976987838745),
 ('Habitable', 0.37219056487083435),
 ('Headstrong', 0.3667587339878082),
 ('uh-oh', 0.3569825291633606),
 ('Hacktivists', 0.35663673281669617)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [12]:
wv['Texas']

array([ 0.9671611 , -1.1006733 ,  2.8143384 ,  0.19919626, -1.035991  ,
       -1.0460798 , -0.6967737 , -0.62599385,  1.6676775 ,  0.90438944,
       -1.1180377 ,  1.1421765 ,  2.0661192 ,  0.73734117, -2.1729543 ,
       -1.9238145 , -2.7921429 ,  0.21943617,  0.2792165 ,  1.1257012 ,
        2.1870341 , -0.4150519 , -1.3622463 , -0.3938973 ,  0.3917156 ,
        1.4289616 , -1.841687  ,  1.3762488 , -1.0188681 ,  0.7217047 ,
       -0.7227689 ,  1.0465746 , -0.736816  , -1.1818869 , -0.16464749,
        0.22223534, -3.0435958 , -0.36143845, -0.5875495 ,  3.2682614 ,
        1.2533194 ,  0.5692444 ,  0.2376802 ,  1.0704242 , -0.36846995,
        1.158536  ,  1.8577243 , -0.02020357, -2.6748707 , -1.6945562 ,
       -1.1875628 , -0.13511118, -1.5207152 , -0.49891517,  0.77713877,
       -2.9114366 , -0.19117086, -1.6124763 ,  0.9497871 ,  0.9260872 ,
        4.9568424 , -3.2342234 ,  0.60858667, -1.7603083 ,  0.24964213,
        0.43236107, -0.45873615, -1.5285772 , -1.0968387 , -0.50

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [13]:
wv.vectors

array([[-1.1326028e+00,  8.6517501e-01, -1.1101983e-01, ...,
        -1.0782981e+00,  3.3067878e-02,  8.1654501e-01],
       [-2.3167461e-01,  2.3043592e+00, -9.9866110e-01, ...,
        -1.5020815e+00,  5.6056696e-01,  2.2563136e-01],
       [-8.7721831e-01,  9.4745642e-01, -5.7639331e-01, ...,
        -2.8825288e+00,  1.2042499e+00,  1.1591823e+00],
       ...,
       [ 1.3990256e-02, -8.4826082e-02,  2.3898192e-02, ...,
         7.6557755e-02, -4.1207086e-02, -4.2469721e-02],
       [-2.5304563e-02,  1.7782511e-02,  7.7202991e-02, ...,
         5.9098350e-03, -3.3666808e-02,  1.2883488e-02],
       [-8.4535154e-03, -9.2643984e-02, -1.7281730e-03, ...,
         3.2729767e-02, -8.4797591e-03, -1.1128027e-01]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [14]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6045567989349365),
 ('crown', 0.5999029874801636),
 ('revival', 0.588080644607544),
 ('queen', 0.5864263772964478),
 ('brunette', 0.5824279189109802),
 ('reminiscent', 0.5718775987625122),
 ('Merchant', 0.5506206750869751),
 ('villain', 0.5423927307128906),
 ('flower', 0.5421411395072937),
 ('diva', 0.5395985841751099)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!